In [18]:
import pandas as pd
import numpy as np
from pathlib import Path
import re

In [19]:
# Notice lines 48 to 52 are for adjusting the output. So depending on whether you want only percentage corrects or percentage correct knowing what is the denaminator comment or uncomment those lines! 
result_dict={} # the final dictionary that will be turned into excel file
result_dict['Participants']=[] # we define a column called 'participants' which will contain participants names
for file in Path('js\\data_online').rglob('*.json'): # As the code is not in the data science folder anymore we don't need to use C:\\Users\\zhaleh\\OneDrive\\phd\\PhD project\\Codes\\Python-JavaScript\\convert_folder_csv_excel\\ to find the folder
    dd=pd.read_json(f'js\\data_online\\{file.name}')
    name=int(re.search(r'\d+',file.name).group()) # get only the number out of the json files
    
    VisReg=dd.iloc[0,0] # gets the visual regular dictionary which is located at the fist row and first column of the json file
    VisIrreg=dd.iloc[0,1]
    VisSD=dd.iloc[0,2]
    AudReg=dd.iloc[0,4]
    AudIrreg=dd.iloc[0,3]
    AudSD=dd.iloc[0,5]
    visreg=pd.DataFrame(VisReg) # turns the dictionary into a dataframe
    visreg.name='VisReg' # we assign a name to the dataframe which will used later for naming columns of the final excel file. 
    visirreg=pd.DataFrame(VisIrreg)
    visirreg.name='VisIrreg'
    vissd=pd.DataFrame(VisSD)
    vissd.name='VisSD'
    audreg=pd.DataFrame(AudReg)
    audreg.name='AudReg'
    audirreg=pd.DataFrame(AudIrreg)
    audirreg.name='AudIrreg'
    audsd=pd.DataFrame(AudSD)
    audsd.name='AudSD'
    
#     print(vissd.groupby('intervalDifference').size())
    List=[visreg, visirreg, vissd, audreg, audirreg, audsd] # this helps to do the same thing for all blocks. Saves time by not writing the code for each block separately
    for i in List: 
        i['userAnswer']=i['userAnswer'].replace('g','4') # sometimes participants has false alarms meaning that they press G instead of 1, 2, 3 and as G is string not number, it causes some problems later in code so I change it to 4 so it doesn't make problem in division later in the code.
        i['userAnswer']=i['userAnswer'].astype(int)      # this column is string instead of number so I correct it this way.    
        intDif=i['intervalDifference'].unique().tolist() # gives me a list of interval differences (one from each case). !!! Weird thig about this is that in single durations it gives me 0.6000000000000001 instead of 0.6 and if I try to fix it with round(1).unique() instead of unique() the result in the excel file would be all 0 so I just leave it there and correct the excel file manually!
        spd=i['speed'].unique().tolist() # [1,2]
        
        for k in intDif: 
            for p in spd:
                intsp=0
                correct=0
                for index,row in i.iterrows():
                    if row['intervalDifference']==k:
                        if row['speed']==p:
                            intsp+=1
                            if row['correctAnswer']==row['userAnswer']:
                                correct+=1
                ratio = round(correct/intsp,2) if intsp!=0 else np.nan # to prevent the problem of division by zero which will give us error! In the final excel 0/0 would be an empty cell because the meaning of 0/0 is different from 0/1 or 0/3. these people (0/0) have got no question to answer so we cannot include them in taking the average!
                key = f'{i.name}/{k}/{p}' # column names
                
                if key not in result_dict: # this part of the code prevents from creating many many dictionaries and instead creates only one dictionary by chaecking for key name
                    result_dict[key] = [ratio]
                    #result_dict[key] = [f'{ratio}/{intsp}']
                else:
                    result_dict[key].append(ratio)
                    #result_dict[key].append(f'{ratio}/{intsp}')
    result_dict['Participants'].append(name) # before looping to the next json file it assigns the name of the participant to the "Participants" column

ddd=pd.DataFrame(result_dict)
ddd.to_excel('IntervalDifSpeed.xlsx',index=False)
print()



In [13]:
# This is the first code I wrote and it works only for one json file and one condition (VisReg)
dd=pd.read_json('Processed Preprocessed10059.json')
VisReg=dd.iloc[0,2]
d=pd.DataFrame(VisReg)
d['userAnswer']=d['userAnswer'].replace('g','4')
d['userAnswer']=d['userAnswer'].astype(int)
############################
participants=[]
int1sp1=0
int1sp2=0
int2sp1=0
int2sp2=0
correct11=0
correct12=0
correct21=0
correct22=0
for index,row in d.iterrows():
    if row['intervalDifference']==1:
        if row['speed']==1:
            int1sp1+=1
            if row['correctAnswer']==row['userAnswer']:
                correct11+=1    
        elif row['speed']==2:
            int1sp2+=1
            if row['correctAnswer']==row['userAnswer']:
                correct12+=1
    elif row['intervalDifference']==2:
        if row['speed']==1:
            int2sp1+=1
            if row['correctAnswer']==row['userAnswer']:
                correct21+=1
        elif row['speed']==2:
            int2sp2+=1
            if row['correctAnswer']==row['userAnswer']:
                correct22+=1
                
ratio11=round(correct11/int1sp1, 2) if int1sp1!= 0 else 0
ratio12=round (correct12/int1sp2,2)if int1sp2!= 0 else 0
ratio21=round(correct21/int2sp1,2) if int2sp1!= 0 else 0
ratio22=round(correct22/int2sp2,2) if int2sp2!= 0 else 0
col1=f'VisSD/0.6/0.23'
col2=f'VisSD/0.6/0.25'
col3=f'VisReg/2/0.23'
col4=f'VisReg/2/0.25'

tmp={col1:ratio11,
    col2:ratio12,
    col3:ratio21,
    col4:ratio22}
participants.append(tmp)
finaldf=pd.DataFrame(participants)
finaldf.to_excel('0000000.xlsx',index=False)
print(d['intervalDifference'].unique().tolist())
dd
print(d.groupby('intervalDifference').size())


[0.4, 0.6000000000000001, 1.0, 2.5, 2.0, 0.5, 3.1, 3.0, 3.5, 2.1, 1.5, 1.6, 2.6]
intervalDifference
0.4    2
0.5    6
0.6    2
1.0    8
1.5    4
1.6    2
2.0    4
2.1    2
2.5    4
2.6    2
3.0    2
3.1    2
3.5    2
dtype: int64
